# Deploy Model using a Script

This Notebook shows how to deploy a model using a custom Python scoring script. This scoring script can be found in `scripts/score.py` and it contains both an `init()` and `run()` method.

The `init()` method is used to load the model from disk, deserialize it and make it available to generate predictions. The `run()` method should process the JSON data provided with each request, generate predictions using the model and return them (in a JSON serializable structure).

This Notebook only shows code to deploy the model on your local machine. Note that Docker must be installed on your machine in order to run this Notebook! Virtual Machines created by AzureML have Docker installed by default.


In [1]:
import requests

from azureml.core import Workspace, Environment, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice

In [2]:
# Get the model
ws = Workspace.from_config()
model = Model(ws, "house_prices_linear_model")

In [3]:
# Set required Python packages
deps = CondaDependencies()
deps.set_python_version("3.8")
deps.add_conda_package("numpy")
deps.add_conda_package("pandas")
deps.add_conda_package("joblib")
deps.add_conda_package("scikit-learn>=0.24")

In [4]:
# Create environment
env = Environment(name='deploy_environment')
env.python.conda_dependencies = deps

In [5]:
# Create configuration settings
inference_config = InferenceConfig(
    environment=env,
    entry_script='scripts/score.py',
)

# Note: Deploying to the local machine; requires Docker to be installed!
deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [6]:

# Build the (local) Docker image and deploy it as a webservice
# Note: This step can take quite a while if the Docker image does not yet exist!
service = Model.deploy(ws, "house-prices-local", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

ModelNotFound: Model with id house_prices_linear_model:4 not found in provided workspace



Generating Docker build context.
Package creation Succeeded
Logging into Docker registry fa8271ca7fb0421488a7b196f8d44ed7.azurecr.io
Logging into Docker registry fa8271ca7fb0421488a7b196f8d44ed7.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM fa8271ca7fb0421488a7b196f8d44ed7.azurecr.io/azureml/azureml_c0b7bed26005472ec5456896c31c10d6
 ---> 8ee53d0f4678
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> a7e339145c8c
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjMyOTU0OGY1LTg1OWMtNGYyYS1hODg0LTQ3MjcwNjEwNmZiYiIsInJlc291cmNlR3JvdXBOYW1lIjoiYXp1cmVtbC1wb2MtbHVrYXMiLCJhY2NvdW50TmFtZSI6ImF6dXJlbWwtcG9jLWx1a2FzIiwid29ya3NwYWNlSWQiOiJmYTgyNzFjYS03ZmIwLTQyMTQtODhhNy1iMTk2ZjhkNDRlZDcifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 1752b986acca
 ---> 7e1dc672bcab
Step 4/5 : RUN mv '/var/azureml-app/tmp_ksu5x4g.py' /var/azureml-app/main.py
 ---> Running in

## Consume Model Endpoint (local)

In [7]:
# Set the endpoint URL
# Note: The port number is specified on the LocalWebservice deployment configuration above.
endpoint = "http://localhost:6789/score"

In [8]:
# Create features for 2 houses
# Note: columns are rooms, house_m2, garden_m2
data = {
    "data": [
        [7, 100, 200],
        [4, 70, 10]
    ]
}

In [9]:
headers = {"Content-Type": "application/json"}
resp = requests.post(endpoint, json=data, headers=headers)

In [10]:
# Get predictions for the 2 houses
resp.json()

[439.4334561066371, 160.14979953503968]